#Diffie-Hellman Key Exchange between notebooks

This notebook has a prelab and two parts with two questions each. Part 1 receives a message from Alice. In Part 2, pick standard values for DHKE parameters for Internet Key Exchange based on from [the Request for Comments (RFC) 3526 standards document](https://www.ietf.org/rfc/rfc3526.txt). If you are new to Colab, see [the introduction](https://colab.research.google.com).

In [ ]:
#         _             _     _
#        | |           | |   | |
#     ___| |_ __ _ _ __| |_  | |__   ___ _ __ ___
#    / __| __/ _` | '__| __| | '_ \ / _ \ '__/ _ \
#    \__ \ || (_| | |  | |_  | | | |  __/ | |  __/
#    |___/\__\__,_|_|   \__| |_| |_|\___|_|  \___|

# Run this entire code block first. It does the following:
#   - Set up the host running the Python environment
#   - Set up the Python environment itself
#   - Define the communication channel between the notebook

# Install dependencies to the Jupyter host filesystem
!pip3 install magic-wormhole twisted-ipython >/dev/null

# Import Python support for magic wormhole
import wormhole
from wormhole.cli import public_relay

# Define magic wormhole static variables
appid = "cis.highline.edu" #namespace agreed on for sharing data
relay_url = public_relay.RENDEZVOUS_RELAY #use the publicly available wormhole

# Communication between notebooks is asynchronous
# magic wormhole uses Twisted Internet for managing asynch threads
from twisted.internet import reactor
%load_ext twisted_ipython
%autoawait twisted

# Define a function that can poll for a message from a specific
# code published at the given wormhole appid
async def get_msg(code=None):
  w = wormhole.create(appid, relay_url, reactor)

  if code is None:
    w.allocate_code()
    code = await w.get_code()
  else:
    w.set_code(code)

  print(f"Getting message from wormhole {code}")

  msg = await w.get_message()
  result = await w.close()
  print(f"Connection closed: {result}")

  return msg

# Define a function that can push a message to specific code
# published at the given wormhole appid
async def send_msg(msg,code=None):
  w = wormhole.create(appid, relay_url, reactor)

  if code is None:
    w.allocate_code()
    code = await w.get_code()
  else:
    w.set_code(code)

  print(f"Sending message to wormhole {code}")

  versions = await w.get_versions()
  w.send_message(msg)
  result = await w.close()
  print(f"Connection closed: {result}")

  return code

# Define a simple function to XOR a secret key with a message plaintext
def xor(string, key):
  if isinstance(string,str) : string = bytearray(string, "utf-8")

  key_bytes = bytearray(str(key), "utf-8")
  encrypted_bytes = bytearray(len(string))

  # Loop through each byte of the string
  for i in range(len(string)):
    # Use modulo to ensure key wraps around if it's longer
    key_index = i % len(key_bytes)
    # XOR each byte with the corresponding byte from the key
    encrypted_bytes[i] = string[i] ^ key_bytes[key_index]

  return encrypted_bytes

## Prelab
Read through the code block above and answer the following questions.

###Question A

Where is the machine that runs the Python code in this notebook?

Answer:

###Question B

How can the XOR operation be used to encrypt data?

Answer:

###Question C

What kind of tunnel does this notebook establish in order to communicate with another notebook?

Answer:

##Part 1
### Alices starts
Bob listens on the wormhole address and generates a private key based on the parameters passed through the tunnel.

In [ ]:
import base64
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import dh

print("What is the wormhole code?")
code = input()

# Create a wormhole and wait for a message that contains Alice's pubkey
alice_trusted = serialization.load_pem_public_key(await get_msg(code))

# Trust Alice's key and generate a key based on Alice's parameters
g = alice_trusted.public_numbers().parameter_numbers.g
p = alice_trusted.public_numbers().parameter_numbers.p

parameters = dh.DHParameterNumbers(p, g).parameters()
bob = parameters.generate_private_key()

pem = bob.public_key().public_bytes(
  encoding=serialization.Encoding.PEM,
  format=serialization.PublicFormat.SubjectPublicKeyInfo
)

await send_msg(pem,code)

s = bob.exchange(alice_trusted)
print(f"Compute the shared secret as s=(A^b) mod p={s.hex()}")

secret_message = await get_msg(code)

print(xor(base64.b64decode(secret_message),s).decode('utf-8'))

###Question 1
How can the shared secret, s, be used for cryptograpy?

Answer:

###Question 2
How many bytes is the shared secret, s?

In [ ]:
# Answer:


##Part 2
###Bob starts
Bob initiates sending a message. He will need to tell Alice which wormhole address to monitor.

In [ ]:
# Write a message to Alice. This message is a secret and so needs to be
# encrypted on the wire.
msg = b'''
Edit this response....
'''

The function below is missing the generator and prime parameters. Review [RFC 3526](https://www.ietf.org/rfc/rfc3526.txt) with your partner to pick values from the standard.

In [ ]:
import base64
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization

# Review ("RFC 3526")[https://www.ietf.org/rfc/rfc3526.txt]
# Select apprpriate parameters and set the two lines below:
generator =   #Add the standard generator value
prime = """

""" #Add the standard prime number as a hex string to the line above

if isinstance(prime, str):
  prime = int("".join(prime.split()), 16)

parameters = dh.DHParameterNumbers(p=prime,g=generator).parameters()
bob = parameters.generate_private_key()

pem = bob.public_key().public_bytes(
  encoding=serialization.Encoding.PEM,
  format=serialization.PublicFormat.SubjectPublicKeyInfo
)

code = await send_msg(pem,code=None)

alice_trusted = serialization.load_pem_public_key(await get_msg(code))

s = bob.exchange(alice_trusted)
print(f"Compute the shared secret as s=(B^a) mod p={s.hex()}")

await send_msg(base64.b64encode(xor(msg,s)),code)


###Question 3
How many bytes is this new shared secret?

In [ ]:
# Answer:


###Question 4
What are the advantages and disadvantages of larger shared secrets?

Answer: